In [1]:
import pandas as pd
import numpy as np
import datetime
import copy

In [2]:
# 案例一（行转列）
# 读取数据集
df1 = pd.read_excel('./示例数据.xlsx',sheet_name='data1')
df1

,value
0,"查询,入库,出库,移库,盘点,设置"


In [3]:
# 方法一：使用 explode 方法
df1_result = df1.value.str.split(',').explode('value')
df1_result

0    查询
1    入库
2    出库
3    移库
4    盘点
5    设置
Name: value, dtype: object

In [4]:
# 案例二（时间相加填充）
# 读取数据集
df2 = pd.read_excel('./示例数据.xlsx',sheet_name='data2')
df2

,产品,工序,加工顺序,加工时间,计划开始时间
0,A,备布,1,120,2021-02-20 10:00:00
1,A,染色,2,480,NaT
2,A,烘干,3,120,NaT
3,A,晒布,4,60,NaT
4,A,定型,5,120,NaT
5,A,检验,6,60,NaT
6,A,入库,7,60,NaT
7,B,备布,1,120,2021-02-21 00:00:00
8,B,预定型,2,180,NaT
9,B,染色,3,480,NaT


In [5]:
# 向上填充缺失日期
df2_copy = copy.deepcopy(df2)
df2_copy = df2_copy.fillna(method='ffill')
df2_copy

,产品,工序,加工顺序,加工时间,计划开始时间
0,A,备布,1,120,2021-02-20 10:00:00
1,A,染色,2,480,2021-02-20 10:00:00
2,A,烘干,3,120,2021-02-20 10:00:00
3,A,晒布,4,60,2021-02-20 10:00:00
4,A,定型,5,120,2021-02-20 10:00:00
5,A,检验,6,60,2021-02-20 10:00:00
6,A,入库,7,60,2021-02-20 10:00:00
7,B,备布,1,120,2021-02-21 00:00:00
8,B,预定型,2,180,2021-02-21 00:00:00
9,B,染色,3,480,2021-02-21 00:00:00


In [6]:
# 使用自定义函数，做时间的加法
df2_copy['分组累加-加工时间'] = df2_copy.groupby(['产品'])['加工时间'].cumsum()
df2_copy

,产品,工序,加工顺序,加工时间,计划开始时间,分组累加-加工时间
0,A,备布,1,120,2021-02-20 10:00:00,120
1,A,染色,2,480,2021-02-20 10:00:00,600
2,A,烘干,3,120,2021-02-20 10:00:00,720
3,A,晒布,4,60,2021-02-20 10:00:00,780
4,A,定型,5,120,2021-02-20 10:00:00,900
5,A,检验,6,60,2021-02-20 10:00:00,960
6,A,入库,7,60,2021-02-20 10:00:00,1020
7,B,备布,1,120,2021-02-21 00:00:00,120
8,B,预定型,2,180,2021-02-21 00:00:00,300
9,B,染色,3,480,2021-02-21 00:00:00,780


In [7]:
# 时间 加 分钟
df2_copy['时间相加'] = df2_copy['计划开始时间'] + pd.to_timedelta(df2_copy['分组累加-加工时间'],unit='m')
df2_copy

,产品,工序,加工顺序,加工时间,计划开始时间,分组累加-加工时间,时间相加
0,A,备布,1,120,2021-02-20 10:00:00,120,2021-02-20 12:00:00
1,A,染色,2,480,2021-02-20 10:00:00,600,2021-02-20 20:00:00
2,A,烘干,3,120,2021-02-20 10:00:00,720,2021-02-20 22:00:00
3,A,晒布,4,60,2021-02-20 10:00:00,780,2021-02-20 23:00:00
4,A,定型,5,120,2021-02-20 10:00:00,900,2021-02-21 01:00:00
5,A,检验,6,60,2021-02-20 10:00:00,960,2021-02-21 02:00:00
6,A,入库,7,60,2021-02-20 10:00:00,1020,2021-02-21 03:00:00
7,B,备布,1,120,2021-02-21 00:00:00,120,2021-02-21 02:00:00
8,B,预定型,2,180,2021-02-21 00:00:00,300,2021-02-21 05:00:00
9,B,染色,3,480,2021-02-21 00:00:00,780,2021-02-21 13:00:00


In [8]:
# 在组中下移一行值
df2_copy['时间下移'] = df2_copy.groupby(['产品'])['时间相加'].shift(1)
df2_copy

,产品,工序,加工顺序,加工时间,计划开始时间,分组累加-加工时间,时间相加,时间下移
0,A,备布,1,120,2021-02-20 10:00:00,120,2021-02-20 12:00:00,NaT
1,A,染色,2,480,2021-02-20 10:00:00,600,2021-02-20 20:00:00,2021-02-20 12:00:00
2,A,烘干,3,120,2021-02-20 10:00:00,720,2021-02-20 22:00:00,2021-02-20 20:00:00
3,A,晒布,4,60,2021-02-20 10:00:00,780,2021-02-20 23:00:00,2021-02-20 22:00:00
4,A,定型,5,120,2021-02-20 10:00:00,900,2021-02-21 01:00:00,2021-02-20 23:00:00
5,A,检验,6,60,2021-02-20 10:00:00,960,2021-02-21 02:00:00,2021-02-21 01:00:00
6,A,入库,7,60,2021-02-20 10:00:00,1020,2021-02-21 03:00:00,2021-02-21 02:00:00
7,B,备布,1,120,2021-02-21 00:00:00,120,2021-02-21 02:00:00,NaT
8,B,预定型,2,180,2021-02-21 00:00:00,300,2021-02-21 05:00:00,2021-02-21 02:00:00
9,B,染色,3,480,2021-02-21 00:00:00,780,2021-02-21 13:00:00,2021-02-21 05:00:00


In [9]:
# 匹配两表
df2_merge = pd.merge(df2_copy,df2,on=['产品','工序','加工顺序'],how='left')
df2_merge

,产品,工序,加工顺序,加工时间_x,计划开始时间_x,分组累加-加工时间,时间相加,时间下移,加工时间_y,计划开始时间_y
0,A,备布,1,120,2021-02-20 10:00:00,120,2021-02-20 12:00:00,NaT,120,2021-02-20 10:00:00
1,A,染色,2,480,2021-02-20 10:00:00,600,2021-02-20 20:00:00,2021-02-20 12:00:00,480,NaT
2,A,烘干,3,120,2021-02-20 10:00:00,720,2021-02-20 22:00:00,2021-02-20 20:00:00,120,NaT
3,A,晒布,4,60,2021-02-20 10:00:00,780,2021-02-20 23:00:00,2021-02-20 22:00:00,60,NaT
4,A,定型,5,120,2021-02-20 10:00:00,900,2021-02-21 01:00:00,2021-02-20 23:00:00,120,NaT
5,A,检验,6,60,2021-02-20 10:00:00,960,2021-02-21 02:00:00,2021-02-21 01:00:00,60,NaT
6,A,入库,7,60,2021-02-20 10:00:00,1020,2021-02-21 03:00:00,2021-02-21 02:00:00,60,NaT
7,B,备布,1,120,2021-02-21 00:00:00,120,2021-02-21 02:00:00,NaT,120,2021-02-21 00:00:00
8,B,预定型,2,180,2021-02-21 00:00:00,300,2021-02-21 05:00:00,2021-02-21 02:00:00,180,NaT
9,B,染色,3,480,2021-02-21 00:00:00,780,2021-02-21 13:00:00,2021-02-21 05:00:00,480,NaT


In [10]:
# 将 NAT 日期替换为另一列中的数据
df2_merge['时间下移'] = df2_merge['时间下移'].combine_first(df2_merge['计划开始时间_y'])
# df2_merge['时间下移'] = np.where(df2_merge['时间下移'].notnull(),df2_merge['时间下移'],df2_merge['计划开始时间_y'])
df2_merge

,产品,工序,加工顺序,加工时间_x,计划开始时间_x,分组累加-加工时间,时间相加,时间下移,加工时间_y,计划开始时间_y
0,A,备布,1,120,2021-02-20 10:00:00,120,2021-02-20 12:00:00,2021-02-20 10:00:00,120,2021-02-20 10:00:00
1,A,染色,2,480,2021-02-20 10:00:00,600,2021-02-20 20:00:00,2021-02-20 12:00:00,480,NaT
2,A,烘干,3,120,2021-02-20 10:00:00,720,2021-02-20 22:00:00,2021-02-20 20:00:00,120,NaT
3,A,晒布,4,60,2021-02-20 10:00:00,780,2021-02-20 23:00:00,2021-02-20 22:00:00,60,NaT
4,A,定型,5,120,2021-02-20 10:00:00,900,2021-02-21 01:00:00,2021-02-20 23:00:00,120,NaT
5,A,检验,6,60,2021-02-20 10:00:00,960,2021-02-21 02:00:00,2021-02-21 01:00:00,60,NaT
6,A,入库,7,60,2021-02-20 10:00:00,1020,2021-02-21 03:00:00,2021-02-21 02:00:00,60,NaT
7,B,备布,1,120,2021-02-21 00:00:00,120,2021-02-21 02:00:00,2021-02-21 00:00:00,120,2021-02-21 00:00:00
8,B,预定型,2,180,2021-02-21 00:00:00,300,2021-02-21 05:00:00,2021-02-21 02:00:00,180,NaT
9,B,染色,3,480,2021-02-21 00:00:00,780,2021-02-21 13:00:00,2021-02-21 05:00:00,480,NaT


In [11]:
# 最终结果
df2_result = df2_merge[['产品','工序','加工顺序','加工时间_x','时间下移']]
df2_result = df2_result.rename({'加工时间_x':'加工时间','时间下移':'计划开始时间'})
df2_result

,产品,工序,加工顺序,加工时间_x,时间下移
0,A,备布,1,120,2021-02-20 10:00:00
1,A,染色,2,480,2021-02-20 12:00:00
2,A,烘干,3,120,2021-02-20 20:00:00
3,A,晒布,4,60,2021-02-20 22:00:00
4,A,定型,5,120,2021-02-20 23:00:00
5,A,检验,6,60,2021-02-21 01:00:00
6,A,入库,7,60,2021-02-21 02:00:00
7,B,备布,1,120,2021-02-21 00:00:00
8,B,预定型,2,180,2021-02-21 02:00:00
9,B,染色,3,480,2021-02-21 05:00:00


In [12]:
# 案例三（输出部门分数前两名的姓名和分数）
# 读取数据集
df3_1 = pd.read_excel('./示例数据.xlsx',sheet_name='data3_1')
df3_1

,编号,姓名,分数,部门
0,1,甲,85,A
1,2,乙,80,B
2,3,丙,60,B
3,4,丁,90,A
4,5,戊,69,A
5,6,己,85,A
6,7,庚,70,A


In [13]:
df3_2 = pd.read_excel('./示例数据.xlsx',sheet_name='data3_2')
df3_2

,部门,部门名
0,A,IT
1,B,销售


In [14]:
# 分组排序
df3_1 = df3_1.sort_values(by=['部门','分数','姓名'],ascending=[True,False,True])
df3_1

,编号,姓名,分数,部门
3,4,丁,90,A
5,6,己,85,A
0,1,甲,85,A
6,7,庚,70,A
4,5,戊,69,A
1,2,乙,80,B
2,3,丙,60,B


In [15]:
# 降序组内排名
df3_1['降序组内排名'] = df3_1['分数'].groupby(df3_1['部门']).rank(method='dense',ascending=False).astype(int)
df3_1

,编号,姓名,分数,部门,降序组内排名
3,4,丁,90,A,1
5,6,己,85,A,2
0,1,甲,85,A,2
6,7,庚,70,A,3
4,5,戊,69,A,4
1,2,乙,80,B,1
2,3,丙,60,B,2


In [16]:
# 匹配 部门名
df3_merge = pd.merge(df3_1,df3_2,on='部门',how='left')
df3_merge = df3_merge[df3_merge['降序组内排名'] <= 3]
df3_result = df3_merge[['部门名','姓名','分数']]
df3_result

,部门名,姓名,分数
0,IT,丁,90
1,IT,己,85
2,IT,甲,85
3,IT,庚,70
5,销售,乙,80
6,销售,丙,60
